In [ ]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
template = "plotly_dark"

In [ ]:
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # * 1e-6 # €/MWh -> €/Wh

In [ ]:
def load_results(dir):
    res = {}
    files = [f for f in os.listdir(dir) if f.endswith(".parquet")]
    for file in files:
        name, _ = os.path.splitext(file)
        res[name] = pd.read_parquet(dir + file)

    return res

In [ ]:
def exctract_value(string, key):
    match = re.search(fr"{key}=([\d.]+)", string)
    if match:
        value = match.group(1)
    return value

In [ ]:
def calc_fec(df):
    power = df["power_sim"]
    power_pos = power[power > 0].sum() * (1 / 60)
    power_neg = power[power < 0].abs().sum() * (1 / 60)
    return (power_pos + power_neg) / 2 / 180e3

In [ ]:
def calc_roundtrip_efficiency(res):
    p = res["power_sim"]
    e_pos = p[p > 0].abs().sum() * 0.25  # Wh
    e_neg = p[p < 0].abs().sum() * 0.25  # Wh

    delta_soc = res["soc_sim"].iloc[-1] - res["soc_sim"].iloc[0]
    delta_e = delta_soc * 180e3  # Wh

    return abs(e_neg) / (e_pos - delta_e)

In [ ]:
# def calc_roundtrip_efficiency2(res):
#     p = res["power_sim"]
#     e_pos = p[p > 0].abs().sum()  # Wh
#     l_pos = res.loc[p > 0, "converter_losses"].sum() + res.loc[p > 0, "battery_losses"].sum()
#     eff_pos = (e_pos - l_pos) / (e_pos)
    
#     e_neg = p[p < 0].sum()  # Wh
#     l_neg = res.loc[p < 0, "converter_losses"].sum() + res.loc[p < 0, "battery_losses"].sum()
#     eff_neg = (e_neg) / (e_neg - l_neg) 

#     return eff_pos #* eff_neg

In [ ]:
def calc_revenue(df, price):
    price = price.resample("1Min").ffill()
    df = df.join(price)
    return -1 * sum(df["power_sim"] * df["Intraday Continuous 15 minutes ID1-Price"]) * (1/60) * 1e-6  # W -> MWh

In [ ]:
def analyse_results(res, price):
    out = pd.DataFrame()
    for (id, df) in res.items():
        data = dict(
            r = exctract_value(id, "r"),
            eff = exctract_value(id, "eff"),
            rev = calc_revenue(df, price),
            fec = calc_fec(df),
            reff = calc_roundtrip_efficiency(df),
        )
        out = pd.concat([out, pd.DataFrame(data=[data])])

    return out

In [ ]:
def plot_timeseries(df, **kwargs):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for id in ["soc_sim", "soc_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=1, col=1)

    for id in ["power_sim", "power_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=2, col=1)
    
    return fig

In [ ]:
dir = "results/"
res = load_results(dir)

In [ ]:
price = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")

In [ ]:
# plot_timeseries(res["2021 LP r=1.0 eff=0.95"], height=800, template=template)

In [ ]:
df = analyse_results(res, price)

In [ ]:
df["fec"] =  df["fec"] / df["fec"].iloc[0]
df["rev"] =  df["rev"] / df["rev"].iloc[0]
df

In [ ]:
fig, ax = plt.subplots()

for eff, group in df.groupby("eff"):
    ax.scatter(x=group["reff"], y=group["rev"], label=eff)

ax.legend()
# fig